## Taxi-v3 с использованием Q-Learning

# Задача:
Есть 4 места (помечены разными буквами), и ваша задача - забрать пассажира в одном месте и высадить его в другом. Вы получаете +20 баллов за успешный переход и теряете 1 балл за каждый временной шаг. Также существует штраф в размере 10 баллов за незаконную посадку и высадку.

# Предварительная установка

In [1]:
import numpy as np
import gym
import random
from tqdm import tqdm

In [3]:
# Создание среды
env = gym.make('Taxi-v3')
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [8]:
# Создаем Q-таблицу
action_size = env.action_space.n
print(f"Общее количество возможных действий: {action_size}")

state_size = env.observation_space.n
print(f"Всего состояний: {state_size}")

Общее количество возможных действий: 6
Всего состояний: 500


In [7]:
q_table = np.zeros((state_size, action_size))
print(q_table)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [9]:
# Гиперпараметры
total_episodes = 50000
total_test_episodes = 10
max_steps = 99

alpha = 0.7
gamma = 0.618                

epsilon = 1.0
max_epsilon = 1.0
min_epsilon = 0.01
decay_rate = 0.01

# Алгоритм Q-Learning

In [13]:
# повторяем каждый эпизод
for episode in tqdm(range(total_episodes)):
    
    # Сбрасываем окружающую среду в каждом эпизоде
    state = env.reset()
    # установите флажок, чтобы проверить, завершен ли эпизод или нет
    done = False
    
    # повторяем все шаги, которые агент может предпринять в эпизоде
    for step in range(max_steps): 
        
        # Выбераем действие на основе политики epsilon-greedy
        
        # вероятность выбора иследования
        one_minus_epsilon = random.uniform(0,1)
        
        if one_minus_epsilon > epsilon:
            action = np.argmax(q_table[state,:])
        else:
            action = env.action_space.sample()
            
        # Выполняем это действие, чтобы перейти в следующее состояние и получить награду
        new_state, reward, done, info = env.step(action)
        
        # обновляем Q-таблицу на основе формулы, приведенной в алгоритме
        q_table[state,action] = q_table[state,action] + alpha*(reward + gamma*np.max(q_table[new_state,:]) - q_table[state,action])
        
        # обновляем текущее состояние
        state = new_state
        
        if done:
            break
        
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)

100%|██████████| 50000/50000 [00:24<00:00, 2028.21it/s]


# Тестируем агента

In [14]:
# следим за всеми наградами
rewards = []

for episode in range(total_test_episodes):
    
    state = env.reset()
    done = False
    total_rewards = 0
    print(f"{'*'*80}")
    print(f"Эпизод {episode + 1}:")
    
    for step in range(max_steps):
        
        # визуализация каждого кадра агента
        env.render()
        
        # совершаем действие, которое принесет максимальную ожидаемую будущую награду в этом состоянии
        action = np.argmax(q_table[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        total_rewards += reward
        
        if done:
            # следим за наградами, полученными в каждом эпизоде
            rewards.append(total_rewards)
            print(f"Счет: {total_rewards}")
            break
        
        state = new_state

env.close()
print(f"Средние Вознаграждение: {sum(rewards)/total_test_episodes}")

********************************************************************************
Эпизод 1:
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R:_| : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| :_| : : |
| : : : : |
| | : | : |
|Y| : |B: |
+--